In [ ]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
import numpy as np
from tbdynamics.calibration.utils import (
    get_bcm,
    calculate_covid_diff_quantiles,
    plot_covid_scenarios_comparison,
    plot_output_ranges,
)
import arviz as az
from tbdynamics.inputs import load_targets
from tbdynamics.constants import  quantiles
from estival.sampling import tools as esamp

In [2]:
OUT_PATH = Path.cwd() / 'runs/best2208'
idata = az.from_netcdf(OUT_PATH / 'calib_full_out.nc')
burnt_idata = idata.sel(draw=np.s_[50000:])
idata_extract = az.extract(burnt_idata, num_samples=300)

In [3]:
params = {
    "start_population_size": 2000000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':True
}

In [ ]:
bcm = get_bcm(params, covid_effects)

In [ ]:
# spaghetti = pd.read_hdf(OUT_PATH / 'results.hdf', 'spaghetti')
# quantile_outputs = esamp.quantiles_for_results(spaghetti, quantiles)
# targets = load_targets()

In [ ]:
# quantile_ouputs_diff = calculate_covid_diff_quantiles(params, idata_extract, ['cumulative_diseased', 'cumulative_deaths'],[2021.0,2022.0],1)

In [ ]:
# plot_covid_scenarios_comparison(quantile_ouputs_diff, ['cumulative_diseased', 'cumulative_deaths'], [2021.0,2022.0], "abs")